In [1]:
import json
import random
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:


# Open the JSON file and load its contents into a dictionary
with open('games.json', 'r') as f:
    ngames = json.load(f)
    

# Print the dictionary to verify that it was loaded correctly
print(ngames)

{'Action Game': ['Grand Theft Auto V', 'Call of Duty: Modern Warfare', "Assassin's Creed Valhalla", 'Doom Eternal', 'Metal Gear Solid V: The Phantom Pain', 'Hades', 'Control', 'Nier: Automata', 'Batman: Arkham Knight', 'Devil May Cry 5', 'Resident Evil Village', 'Deathloop', 'Far Cry 6', "Ghost of Tsushima Director's Cut", 'Returnal'], 'Adventure Game': ['The Legend of Zelda: Breath of the Wild', "Uncharted 4: A Thief's End", 'Red Dead Redemption 2', 'The Last of Us Part II', 'Death Stranding', 'Horizon Zero Dawn', "Assassin's Creed Odyssey", 'God of War', 'Tomb Raider', 'Shadow of the Colossus', 'It Takes Two', 'Kena: Bridge of Spirits', 'Life is Strange: True Colors', 'Psychonauts 2', 'The Artful Escape'], 'Role-Playing Game': ['The Witcher 3: Wild Hunt', 'Skyrim', 'Fallout 4', 'Mass Effect 2', 'Final Fantasy VII Remake', 'Persona 5', 'Dragon Age: Inquisition', 'Divinity: Original Sin II', 'Dark Souls III', 'Kingdom Come: Deliverance', 'Tales of Arise', 'Scarlet Nexus', 'Disco Elysiu

In [4]:

def create_transaction_data(num_transactions, max_games_per_transaction,mode):
    data = []

    game_names = [
        'Action Game', 'Adventure Game', 'Role-Playing Game', 'Strategy Game',
        'Simulation Game', 'Sports Game', 'Puzzle Game', 'Racing Game',
        'Shooter Game', 'Platformer Game', 'Indie Game', 'Horror Game'
    ]

    for transaction_id in range(1, num_transactions + 1):
        if max_games_per_transaction <= 2:
            max_games_per_transaction = 2
        num_games = random.randint(2, max_games_per_transaction)
        games = random.sample(game_names, num_games)
        amount = round(random.uniform(10, 100), 2)
        purchase_date = f'2023-09-{random.randint(1, 30)}'
        discount = round(random.uniform(0, 0.3), 2)
        mpay=random.choice(['cash','card','robbery','UPI','Membership'])


        transaction = {
            'TransactionID': transaction_id,
            'Games': games,
            'Amount': amount,
            'PurchaseDate': purchase_date,
            'Discount': discount,
            'Payment_mode': mpay
            
        }

        data.append(transaction)

    
    if mode==1:
        return data
    else:
        # Convert the data list to JSON format
        json_data = json.dumps(data, indent=4)

        # Save the JSON data to a file
        with open('transaction_data.json', 'w') as json_file:
            json_file.write(json_data)
        return json_data


In [5]:
num_transactions = 1000 #int(input("Enter the number of transactions: "))
max_games_per_transaction = 12#int(input("Enter the maximum number of games per transaction: "))
transaction_data = create_transaction_data(num_transactions, max_games_per_transaction,1)

In [6]:
def recom_engine(transaction_data,user_game):
    num_recommendations = 2
    try:
        df = pd.DataFrame(transaction_data)

        oht = pd.get_dummies(df['Games'].apply(pd.Series).stack()).groupby(level=0).sum()
        random_number = random.uniform(0.20, 0.30)
        msc=round(random_number,2)



        frequent_itemsets = apriori(oht, min_support=msc, use_colnames=True)

        rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1.0)


        user_rules = rules[rules['antecedents'] == {user_game}]
        user_rules = user_rules.sort_values(by='lift', ascending=False).head(num_recommendations)


        # recommended_games = list(user_rules['consequents'].values)

        
        recommended_games = [list(game)[0] for game in user_rules['consequents']]
        # print(f"Recommended games for {user_game}: {recommended_games[0]}")
        return recommended_games
        
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")



In [21]:
import numpy as np
import math

def recom_sys():
    outlis=[]
    user_game = '1' #input("Enter a video game: ")
    if user_game=='1':
        user_game='Action Game'
    recom_need = int(input("Enter the number of game recommendations: "))
    n1=recom_need*0.6
    # n2=recom_need*0.4
    # n2=math.ceil(n2) these were planned so that ratio of both generes would be 60:40 , but didnt added this feature till this comment is here :)
    n1=math.floor(n1)
    # print(n1)
    
    recoms=recom_engine(transaction_data,user_game)
    print(recoms)
    # print(recom_list)

    while len(outlis)<recom_need:
        for genere in recoms:
            if len(ngames[genere])>len(ngames[genere][0:n1]):
                outlis+=ngames[genere][0:n1]
            else:
                outlis.append('trash af')
            recoms.remove(genere)
    # recom_list=
    return ((outlis[:recom_need]))

rl=recom_sys()
print('Here are your',len(rl),'recommendations:','\n')
for srn,rec in enumerate(rl):
    print('\t',srn+1,'--> ',rec)

['Puzzle Game', 'Simulation Game']
Here are your 18 recommendations: 

	 1 -->  Portal 2
	 2 -->  Tetris Effect
	 3 -->  The Witness
	 4 -->  Baba Is You
	 5 -->  The Talos Principle
	 6 -->  Lumines Remastered
	 7 -->  Fez
	 8 -->  Monument Valley
	 9 -->  The Room
	 10 -->  Puyo Puyo Tetris
	 11 -->  The Sims 4
	 12 -->  Cities: Skylines
	 13 -->  Planet Coaster
	 14 -->  Kerbal Space Program
	 15 -->  Farming Simulator 19
	 16 -->  Microsoft Flight Simulator
	 17 -->  Euro Truck Simulator 2
	 18 -->  RollerCoaster Tycoon 2
